In [35]:
import pandas as pd

In [36]:
all_divisions = {
    "Premier Main": 424,
    "2": 425,
    "3": 426,
    "4": 427,
    "5": 428,
    "6": 429,
    "7A": 430,
    "7B": 431,
    "8A": 432,
    "8B": 433,
    "9": 434,
    "10": 435,
    "11": 436,
    "12": 437,
    "13A": 438,
    "13B": 439,
    "14": 440,
    "15A": 441,
    "15B": 442,
    "Premier Masters": 443,
    "M2": 444,
    "M3": 445,
    "M4": 446,
    "Premier Ladies": 447,
    "L2": 448,
    "L3": 449,
    "L4": 450,
    }

In [37]:
summary_directory = r"C:\Users\bpali\PycharmProjects\SquashApp\2024-2025\summary_df"

In [38]:
awaiting_results_directory = r"C:\Users\bpali\PycharmProjects\SquashApp\2024-2025\awaiting_results"

In [39]:
# In the the summary directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

summary_dfs = []
for division in all_divisions.keys():
    for week in range(10, 0, -1):
        try:
            summary_df = pd.read_csv(f"{summary_directory}\\week_{week}\\{division}_summary_df.csv")
            summary_df["Division"] = division
            # Create Rank column based on Points column
            summary_df["Rank"] = summary_df["Points"].rank(ascending=False, method="min")
            # Make sure Rank column is an integer
            summary_df["Rank"] = summary_df["Rank"].astype(int)
            # Create column for number of teams in the division
            summary_df["Teams"] = len(summary_df)
            summary_dfs.append(summary_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the summary dataframes into one
summary_df = pd.concat(summary_dfs)

In [40]:
# In the the awaiting_results directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

awaiting_results_dfs = []
for division in all_divisions.keys():
    for week in range(10, 0, -1):
        try:
            awaiting_results_df = pd.read_csv(f"{awaiting_results_directory}\\week_{week}\\{division}_awaiting_results.csv")
            awaiting_results_df["Division"] = division
            awaiting_results_dfs.append(awaiting_results_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the awaiting_results dataframes into one
awaiting_results_df = pd.concat(awaiting_results_dfs)

In [41]:
hkcc = "Hong Kong Cricket Club"

# Filter rows where Team column contains "Hong Kong Cricket Club"
hkcc_summary_df = summary_df[(summary_df["Team"].str.contains(hkcc)) | 
                             (summary_df["Team"].str.contains("hkcc", case=False))]

hkcc_summary_df

,Team,Played,Won,Lost,Points,Rubbers For,Rubbers Against,Division,Rank,Teams
1,Hong Kong Cricket Club 1,3,3,0,9,6,3,Premier Main,2,6
2,Hong Kong Cricket Club 2,3,1,2,6,5,7,2,3,5
6,Hong Kong Cricket Club 3,4,1,3,6,5,15,4,7,8
3,Hong Kong Cricket Club 4,4,3,1,17,14,6,6,4,10
5,Hong Kong Cricket Club 5,3,1,2,8,7,8,7A,5,7
4,Hong Kong Cricket Club 6,3,2,1,10,8,7,7B,4,7
10,HKCC Tuesday Night Rockers,3,0,3,4,4,11,11,11,11
3,Hong Kong Cricket Club 8,4,1,3,11,10,10,15A,4,7
2,Hong Kong Cricket Club M1A,4,1,3,5,4,8,Premier Masters,3,5
4,Hong Kong Cricket Club M1B,2,1,1,3,2,4,Premier Masters,5,5


In [48]:
round(hkcc_summary_df["Won"].sum() / hkcc_summary_df["Played"].sum(), 3)

0.448

In [ ]:
# Filter rows where Home Team column or Away Team column contains "Hong Kong Cricket Club" or "hkcc" (case-insensitive)
hkcc_awaiting_results_df = awaiting_results_df[(awaiting_results_df["Home Team"].str.contains(hkcc, case=False)) | 
                                               (awaiting_results_df["Away Team"].str.contains(hkcc, case=False)) |
                                               (awaiting_results_df["Home Team"].str.contains("hkcc", case=False)) |
                                                (awaiting_results_df["Away Team"].str.contains("hkcc", case=False))]


In [45]:
hkcc_awaiting_results_df

,Home Team,Away Team,Venue,Match Week,Date,Division
0,Kinetic 1,Hong Kong Cricket Club 5,HK Squash Centre,5,2024-10-30,7A
0,Xavier,Hong Kong Cricket Club 6,Cornwall Street Squash Centre,5,2024-10-30,7B
0,Ladies Recreation Club M1,Hong Kong Cricket Club M1B,Ladies Recreation Club,4,2024-10-24,Premier Masters
